In [10]:
!pip install -q darts==0.24 ray==2.3.1 optuna==3.1.0

files = [
    "benchmark_tools.py",
    "model_evaluation.py",
    "optuna_search.py",
    "param_space.py",
]
for f in files:
    url = f"https://raw.githubusercontent.com/unit8co/darts/docs/benchmarks/experiments/{f}"
    !wget --no-cache --backups=1 {url}

In [ ]:
from benchmark_tools import Dataset, experiment, silence_prompt
from darts.models import NaiveSeasonal, NHiTSModel, Prophet, LightGBMModel
from darts.utils import missing_values
import warnings

models = [NaiveSeasonal, NHiTSModel, Prophet]

In [ ]:
from darts.datasets import AirPassengersDataset, WeatherDataset

air_pass_series = missing_values.fill_missing_values(AirPassengersDataset().load())
weather_ds = missing_values.fill_missing_values(WeatherDataset().load().resample("1h"))[
    -1500:
]
weather_past_cov = weather_ds[
    ["p (mbar)", "wv (m/s)", "wd (deg)", "rain (mm)", "raining (s)", "SWDR (W/m²)"]
]
weather_series = weather_ds["T (degC)"]

dataset_air_passengers = Dataset(name="Air passengers", series=air_pass_series)
dataset_weather = Dataset(
    name="Weather", series=weather_series, past_covariates=weather_past_cov
)

In [ ]:
from optuna_search import optuna_search

silence_prompt()
best_params = optuna_search(
    model_class=LightGBMModel, **dataset_air_passengers._asdict(), time_budget=60
)
model = LightGBMModel(**best_params)

train_split = 0.8
train_limit = int(train_split * len(dataset_air_passengers.series))
model.fit(dataset_air_passengers.series[:train_limit])
forecasts = model.historical_forecasts(
    series=dataset_air_passengers.series, start=train_split, retrain=False
)

dataset_air_passengers.series.plot(label="actual")
forecasts.plot(label="prediction")

In [ ]:
results = experiment(
    list_datasets=[dataset_air_passengers, dataset_weather],
    models=models,
    grid_search=False,
    forecast_horizon=0.05,
    repeat=5,
    silent_search=True,
)

print("\n\n", results)